# Image Super-Resolution Using K-SVD


## Importing useful libraries

In [ ]:
#Import Numpy for matrix operations
import numpy as np

#Import the normalize function from preprocessing package from sklearn
from sklearn.preprocessing import normalize

#Import the Orthogonal Matching Pursuit Algorithm from Scikit-Learn, for 
#the implementation of the K-SVD and Approximate K-SVD algorithm
from sklearn.linear_model import orthogonal_mp

#Import the Sparse SVD function from scipy, used in the original SVD algorithm
from scipy.sparse.linalg import svds

#Import the norm function from the linear algebra package of numpy, used in 
#the implementation of K-SVD
from numpy.linalg import norm, inv

#Import the walk function from the os package to perform file operations
from os import walk

from PIL import Image
import matplotlib.pyplot as plt

## Approximate K-SVD

In [ ]:
#Implementation of the Approximate K-SVD algorithm (Rubinstein et. al (2008))
def Approx_K_SVD(signal_set, dictionary_size, sparsity, n_iterations=100):

  #get the number of features (Signal size)
  n_features = signal_set.shape[0]

  #initlialise the dictionary 
  D = np.random.randn(n_features, dictionary_size)
  #normalize the columns of D
  D = normalize(D, norm='l2', axis = 0)
  norm_score = 1000
  iteration = 1
  for iteration in range(n_iterations):
    print("Iteration:", iteration)
    #-----------------------------------------------------------------------
    #                         Spare Coding Step
    #-----------------------------------------------------------------------
    #Here, we run the OMP algorithm on the dictionary and the signal set to
    #obtain the sparse code for the current iteration
 
    sparse_code = orthogonal_mp(D, signal_set, sparsity)
    #loop through every column of D to update
    for i in range(dictionary_size):
      #make the current column zero. 
      #This is a simplification for the code that follows
      D[:, i] = np.zeros((n_features))
      #find the signals that used the column i of the dictionary D
      indices = np.nonzero(sparse_code[i, :])
      if indices[0].size == 0:
        continue

      #----------------------------------------------------------------------
      #                     Dictionary update step
      #----------------------------------------------------------------------
      # Instead of using an SVD decomposition as above, we use an approximate
      # method as proposed in the paper by Rubinstein et. al (2008)

      g = sparse_code[i, indices[0]].T
      temp = signal_set[:, indices[0]] - D.dot(sparse_code[:, indices[0]])
      d = temp.dot(g)
      d = normalize(d.reshape(-1,1), axis=0)
      g = temp.T.dot(d)
      D[:, i] = d.ravel()
      sparse_code[i, indices[0]] = g.T

    print(norm(signal_set-np.matmul(D, sparse_code)))
    norm_score = norm(signal_set-np.matmul(D, sparse_code))/norm(signal_set)
    iteration += 1 
  return D, sparse_code

## Utility functions

In [ ]:
def get_patch(i, j, n, image):
    K = int(np.sqrt(n))
    R = np.zeros((K,K, 3))
    img_data = np.asarray(image)
    for k in range(K):
        for l in range(K):
            R[k,l, :] = img_data[i+k, j+l, :]
        
    return R

In [ ]:
def extract_patches(img_data, patch_size):
    K = int(np.sqrt(patch_size))
    n_patches = int(img_data.shape[0] * img_data.shape[1]/patch_size)
    X = np.zeros((patch_size*3, n_patches))

    for i in range(int(img_data.shape[0]/K)):
        for j in range(int(img_data.shape[1]/K)):
            X[:, i*int(img_data.shape[0]/K)+j] = get_patch(i*K, j*K, patch_size, img_data).flatten()
    return  X

In [ ]:
def recreate_image(patch_array, img_data_shape, patch_size):
    K = int(np.sqrt(patch_size))
    new_img = np.zeros((img_data_shape[0], img_data_shape[1], 3))
    for i in range(int(img_data_shape[0]/K)):
        for j in range(int(img_data_shape[1]/K)):
            new_img[i*K:(i+1)*K, j*K:K*(j+1), :] = patch_array[:, i*int(img_data_shape[0]/K) + j].reshape(K, K, 3)
    return new_img

In [ ]:
def post_process(img_data):
    img_data = np.minimum(img_data, np.full(img_data.shape, 255.0))
    img_data = np.maximum(img_data, np.full(img_data.shape, 0.0))
    return img_data

## Running the K-SVD Algorithm


In [ ]:
# load the image
image = Image.open('image.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()
image = image.resize((1024, 1024))

JPEG
RGB
(1250, 1250)


In [ ]:
downscaled_image = image.resize((512, 512))
downscaled_image.save('downscaled.png')
downscaled_img = np.asarray(downscaled_image).reshape(-1, 1)

In [ ]:
X = extract_patches(np.asarray(downscaled_image), 4)

In [ ]:
D, sparse_code = Approx_K_SVD(X, 200, 10, 5)

Iteration: 0
250.21355177492148
Iteration: 1
246.0441530745861
Iteration: 2
242.99229973031234
Iteration: 3
239.23679444067932
Iteration: 4
235.5723757633997


In [ ]:
new_img = recreate_image(D.dot(sparse_code), downscaled_image.size, 4)

In [ ]:
Y = extract_patches(np.asarray(image), int(image.size[0]*image.size[1]/65536))
temp = inv(np.matmul(sparse_code, sparse_code.T))
D_new = Y.dot(sparse_code.T).dot(temp)
reconstructed = D_new.dot(sparse_code)

new_img = recreate_image(reconstructed, (1024, 1024, 3), int(image.size[0]*image.size[1]/65536))
new_img = post_process(new_img)
Image.fromarray(new_img.astype('uint8')).save('scene_big.png')

(48, 65536)


In [ ]:
Image.fromarray(new_img.astype('uint8')).show()